In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [16]:
import numpy as np
import pandas as pd

import librosa
import torch

from scipy import spatial
import matplotlib.pyplot as plt

import os
import pickle

from pathlib import Path
import tqdm

from IPython.display import Audio
import sys

In [19]:
from towhee import pipe # !pip install towhee towhee.models
import laion_clap # !pip install laion-clap


# Load Files

In [1]:
pwd

'/Users/juliabarnett/Desktop/Northwestern/*Fall 2023/Audio Project/git/exploring-musical-roots'

In [6]:
p_in = 'path/to/music/files/'
#p_in = 'toy-song-folder/mp3s/'

In [7]:
len(os.listdir(p_in))

500

In [8]:
# find all our input files
file_list = [f for f in os.listdir(p_in)]


# CLMR Embeddings

In [9]:
from towhee import pipe, ops, DataCollection

p = (
    pipe.input('path')
        .map('path', 'frame', ops.audio_decode.ffmpeg())
        .map('frame', 'vecs', ops.audio_embedding.clmr())
        .output('path', 'vecs')
)

In [10]:
audio_dict_CLMR = {}

In [11]:
%%time 
# on my machine, this takes ~1 minute for 500 clips
# results is a dictionary (audio_dict_CLMR) with key = filename, value = CLMR embedding
string_start = p_in
num = 0
for file in file_list:
    print(num)
    # try statement here allows you to skip to items you haven't yet embedded if you stop this step midway (if a key exists, you move on to next key)
    try:
        audio_dict_CLMR[file]
    except:
        # this step grabs a CLMR embedding from the towhee CLMR pipeline
        audio_dict_CLMR[file] = DataCollection(p(p_in + file))[0]['vecs']
    num = num + 1

0
1


[W NNPACK.cpp:53] Could not initialize NNPACK! Reason: Unsupported hardware.


2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
27

In [ ]:
# output from here: audio_dict_CLMR

# CLAP Embedding

In [14]:
# quantization
def int16_to_float32(x):
    return (x / 32767.0).astype(np.float32)


In [13]:
def float32_to_int16(x):
    x = np.clip(x, a_min=-1., a_max=1.)
    return (x * 32767.).astype(np.int16)


In [15]:
model = laion_clap.CLAP_Module(enable_fusion=False)
model.load_ckpt() # download the default pretrained checkpoint.


/Users/juliabarnett/opt/miniconda3/envs/towhee_env/lib/python3.7/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/TensorShape.cpp:3191.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequen

Load our best checkpoint in the paper.
The checkpoint is already downloaded
Load Checkpoint...
logit_scale_a 	 Loaded
logit_scale_t 	 Loaded
audio_branch.spectrogram_extractor.stft.conv_real.weight 	 Loaded
audio_branch.spectrogram_extractor.stft.conv_imag.weight 	 Loaded
audio_branch.logmel_extractor.melW 	 Loaded
audio_branch.bn0.weight 	 Loaded
audio_branch.bn0.bias 	 Loaded
audio_branch.patch_embed.proj.weight 	 Loaded
audio_branch.patch_embed.proj.bias 	 Loaded
audio_branch.patch_embed.norm.weight 	 Loaded
audio_branch.patch_embed.norm.bias 	 Loaded
audio_branch.layers.0.blocks.0.norm1.weight 	 Loaded
audio_branch.layers.0.blocks.0.norm1.bias 	 Loaded
audio_branch.layers.0.blocks.0.attn.relative_position_bias_table 	 Loaded
audio_branch.layers.0.blocks.0.attn.qkv.weight 	 Loaded
audio_branch.layers.0.blocks.0.attn.qkv.bias 	 Loaded
audio_branch.layers.0.blocks.0.attn.proj.weight 	 Loaded
audio_branch.layers.0.blocks.0.attn.proj.bias 	 Loaded
audio_branch.layers.0.blocks.0.norm2.we

In [17]:
audio_dict_CLAP = {}

In [18]:
%%time 
# on my machine, this takes ~3 minutes for 500 clips
num = 0
for file in file_list:
    print(num)
    # try statement here allows you to skip to items you haven't yet embedded if you stop this step midway (if a key exists, you move on to next key)
    try:
        audio_dict_CLAP[file]
    except:
        # Get audio embeddings from audio data
        full_path = p_in + "/" + file
        # this step grabs a CLAP embedding from laion_clap library
        audio_embed = model.get_audio_embedding_from_filelist(x = [full_path], use_tensor=False)
        audio_dict_CLAP[file] = audio_embed
    num = num + 1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [20]:
# output from here: audio_dict_CLAP

# Build DataFrame of the Above

In [33]:
flat_df_CLMR = pd.DataFrame(audio_dict_CLMR.items(), columns=['vector_name','vector_embed']).reset_index()
flat_df_CLMR.columns=['vector_id','vector_name','vector_embed']

flat_df_CLAP= pd.DataFrame(audio_dict_CLAP.items(), columns=['vector_name','vector_embed']).reset_index()
flat_df_CLAP.columns=['vector_id','vector_name','vector_embed']

In [34]:
# do these to flatten the vectors for proper format to upload to Pinecone

def flatten_vector_embed(vector_embed):
    return list(vector_embed.flatten())

def grab_song_title(vector_name):
    return vector_name.split("_")[0]


In [35]:
flat_df_CLMR["song_title"] = flat_df_CLMR.apply(lambda row: grab_song_title(row['vector_name']),axis=1)
flat_df_CLMR["flat_vector_embed"] = flat_df_CLMR.apply(lambda row: flatten_vector_embed(row['vector_embed']),axis=1)

flat_df_CLAP["song_title"] = flat_df_CLAP.apply(lambda row: grab_song_title(row['vector_name']),axis=1)
flat_df_CLAP["flat_vector_embed"] = flat_df_CLAP.apply(lambda row: flatten_vector_embed(row['vector_embed']),axis=1)


# Vector cleaning for pinecone specifics

In [76]:
def convert_to_npfloat64(original_array):
    #return np.array(flat_df["flat_vector_embed"][0],dtype=np.float64)
    return np.array(original_array,dtype=np.float64)

def convert_to_npfloat64_to_list(vector_embed_64):
    # list(flat_df["flat_vector_embed_64"][0])
    return list(vector_embed_64)


In [77]:
flat_df_CLMR["flat_vector_embed_64"] = flat_df_CLMR.apply(lambda row: convert_to_npfloat64(row['flat_vector_embed']),axis=1)
flat_df_CLMR["flat_vector_embed_64_list"] = flat_df_CLMR.apply(lambda row: convert_to_npfloat64_to_list(row['flat_vector_embed_64']),axis=1)

flat_df_CLAP["flat_vector_embed_64"] = flat_df_CLAP.apply(lambda row: convert_to_npfloat64(row['flat_vector_embed']),axis=1)
flat_df_CLAP["flat_vector_embed_64_list"] = flat_df_CLAP.apply(lambda row: convert_to_npfloat64_to_list(row['flat_vector_embed_64']),axis=1)


# Build Metadata Df

In [47]:
metadata_folder = "path/to/metadata/in/json/form/"
#metadata_folder = "toy-song-folder/jsons/"

meta_list = [f for f in os.listdir(metadata_folder)]
meta_list = sorted(meta_list)

In [51]:
meta_df = pd.read_json(metadata_folder + "/" + meta_list[0])

for i in range(1, len(meta_list)-1):
    new_row = pd.read_json(metadata_folder + "/" + meta_list[i])
    meta_df = meta_df.append(new_row).reset_index(drop = True)

### Add MetaData

In [79]:
def look_up_metadata(track_id, meta_dataframe, meta_col_interest):
    # track_id: form = spotify:track:id_##,mp3
    # meta_datframe: df of all the metavalues
    # column options = album, artist_names, popularity, release_date, genre
    df_id = track_id.split("_")[0]
    meta_row = meta_df[meta_df['uri'] == df_id].reset_index(drop=True)
    try: 
        return meta_row[meta_col_interest][0]
    except:
        return "unknown"
    #return meta_row[meta_col_interest][0]

In [80]:
meta_df.head()

,name,uri,album,album_uri,artist_names,artist_uris,popularity,playlist_uri,release_date,genre,track_id,duration_ms
0,Fallin' Apart - Close Counters Remix,spotify:track:02TyCVplEGRLcyW9VmzVX7,Fallin' Apart (Remixes),spotify:album:2oxOG8dDqcJc4yJi6JCYHA,Young Franco,spotify:artist:6mK0vAO13gT8jWYANyoXAl,0,spotify:playlist:5fCodh7HOjTe1evi7OjclY,2021-01-15,jazzy dnb,02TyCVplEGRLcyW9VmzVX7,197258.0
1,this is not a love song,spotify:track:02WtbZhLOGOdut68sAs5aB,this is not a love song,spotify:album:0BnaolN5a5sCBUXUJIpA0d,MONA,spotify:artist:1cFDUoQ5wYnwrHsSkwvUHr,14,spotify:playlist:4rWRvJZr15hxgyZ1ozlpXb,2022-11-04,korean jazz,02WtbZhLOGOdut68sAs5aB,201226.0
2,Fools Rush In,spotify:track:02gauNUp4Rr9N4f1EvSx0f,Day By Day,spotify:album:0DQqEx56iwBAsSiz05KjFy,The Four Freshmen,spotify:artist:7eAF64ZiDwK2rDPSrr97D9,35,spotify:playlist:0cRFNuuqOlpXgm2ezbD69y,1994,indie jazz,02gauNUp4Rr9N4f1EvSx0f,156159.0
3,We Have All The Time In The World (feat. The D...,spotify:track:03qBsFKs6aD6SFk8RqpHm9,"Pisnicky, ktere Karel miloval",spotify:album:7DtH75CC7NpoenTvu8A0Av,Various Artists,spotify:artist:0LyfQWJT6nXafLPZqxe9Of,34,spotify:playlist:37i9dQZF1DWT8pEFXwG9EZ,2012-12-14,bossa nova jazz,03qBsFKs6aD6SFk8RqpHm9,193973.0
4,It Could Happen To You,spotify:track:04edq8nmRKQOo2yQLBwgk7,Waltz For Debby,spotify:album:5jMKNvYYHH08quqLK3ylTC,Monica Zetterlund,spotify:artist:7mvvG63CNSY93JWAJ37rnD,33,spotify:playlist:37i9dQZF1DX1C8KR4UJlnr,1964-12-01,trad jazz catala,04edq8nmRKQOo2yQLBwgk7,179666.0


In [82]:
flat_df_CLMR["genre"] = flat_df_CLMR.apply(lambda row: look_up_metadata(row['vector_name'], meta_df, 'genre'),axis=1)
flat_df_CLMR["album"] = flat_df_CLMR.apply(lambda row: look_up_metadata(row['vector_name'], meta_df, 'album'),axis=1)
flat_df_CLMR["name"] = flat_df_CLMR.apply(lambda row: look_up_metadata(row['vector_name'], meta_df, 'name'),axis=1)
flat_df_CLMR["artist"] = flat_df_CLMR.apply(lambda row: look_up_metadata(row['vector_name'], meta_df, 'artist_names'),axis=1)

flat_df_CLAP["genre"] = flat_df_CLAP.apply(lambda row: look_up_metadata(row['vector_name'], meta_df, 'genre'),axis=1)
flat_df_CLAP["album"] = flat_df_CLAP.apply(lambda row: look_up_metadata(row['vector_name'], meta_df, 'album'),axis=1)
flat_df_CLAP["name"] = flat_df_CLAP.apply(lambda row: look_up_metadata(row['vector_name'], meta_df, 'name'),axis=1)
flat_df_CLAP["artist"] = flat_df_CLAP.apply(lambda row: look_up_metadata(row['vector_name'], meta_df, 'artist_names'),axis=1)


In [83]:
def strip_year_from_date(full_date):
    if type(full_date) == int:
        return str(full_date)
    else:
        try:
            return full_date[:4]
        except:
            return "CHECK_THIS"
        

In [84]:
meta_df["year"] = meta_df.apply(lambda row: strip_year_from_date(row['release_date']),axis=1)


In [86]:
flat_df_CLMR["year"] = flat_df_CLMR.apply(lambda row: look_up_metadata(row['vector_name'], meta_df, 'year'),axis=1)
flat_df_CLAP["year"] = flat_df_CLAP.apply(lambda row: look_up_metadata(row['vector_name'], meta_df, 'year'),axis=1)


In [87]:
def strip_vector_clip(vector_name):
    return vector_name.split(".")[0].split("_")[1]

In [88]:
flat_df_CLMR["vector_clip_num"] = flat_df_CLMR.apply(lambda row: strip_vector_clip(row['vector_name']),axis=1)
flat_df_CLAP["vector_clip_num"] = flat_df_CLAP.apply(lambda row: strip_vector_clip(row['vector_name']),axis=1)


In [93]:
def get_triplet_num(vector_name_str):
    return str(int(vector_name_str.split("_")[2].split(".")[0]) + 1)

In [94]:
flat_df_CLMR['embedding_triplet_num'] = flat_df_CLMR.vector_name.apply(get_triplet_num)
flat_df_CLAP['embedding_triplet_num'] = flat_df_CLAP.vector_name.apply(get_triplet_num)


In [95]:
print("unique songs:", len(flat_df_CLMR.song_title.unique()))
print("unique songs:", len(flat_df_CLAP.song_title.unique()))


unique songs: 493
unique songs: 493


In [96]:
# # this is for later to check against metadata once uploaded

# def index_meta_search(vector_embed, index_var):
#     test_quer = index_var.query(
#     vector=vector_embed,
#     top_k=10,
#     #include_values=False,
#       include_metadata=True
#     )
#     return test_quer
    

# Pinecone Setup

In [139]:
import openai

from typing import List, Iterator
import pandas as pd
import numpy as np
import os
#import wget
from ast import literal_eval

# # Pinecone's client library for Python
import pinecone


In [142]:
# This is to upload your vectors
# Models a simple batch generator that make chunks out of an input DataFrame
class BatchGenerator:
    #
    def __init__(self, batch_size: int = 10) -> None:
        self.batch_size = batch_size
    #
    # Makes chunks out of an input DataFrame
    def to_batches(self, df: pd.DataFrame) -> Iterator[pd.DataFrame]:
        splits = self.splits_num(df.shape[0])
        if splits <= 1:
            yield df
        else:
            for chunk in np.array_split(df, splits):
                yield chunk
    #
    # Determines how many chunks DataFrame contains
    def splits_num(self, elements: int) -> int:
        return round(elements / self.batch_size)
    #
    __call__ = to_batches

In [143]:
df_batcher = BatchGenerator(64)

In [ ]:
# can sign up for free here: https://www.pinecone.io/

In [97]:
api_key = "XXXXXX" # put your API key from pinecone here

# CLMR Uploads

In [154]:
# this is free! so we can leave this one running

In [156]:
# Name for the index (must match that on Pinecone website)
# CLMR:
index_name_clmr = 'name-of-clmr-pinecone-index' # free (comes with plan, can have 100k records)
index_env_clmr = 'name-of-pinecone-environment' # free (comes with plan)


In [158]:
pinecone.init(api_key=api_key, 
              environment=index_env_clmr)

In [159]:
index_clmr = pinecone.Index(index_name=index_name_clmr)


In [160]:
pinecone.list_indexes()


['clmr-small-index']

### Upload vectors to Pinecone

In [161]:
batch_id = 0
for batch_df in df_batcher(flat_df_CLMR):
    #print(batch_df)
    print("batch:", batch_id)
    batch_id = batch_id + 1
    index_clmr.upsert(vectors=list(zip(batch_df["vector_name"],batch_df["flat_vector_embed_64_list"])))

batch: 0
batch: 1
batch: 2
batch: 3
batch: 4
batch: 5
batch: 6
batch: 7


### Upload their metadata as well: 

In [162]:
failed_list_update_metadata_CLMR = []
# it sometimes fails due to weird characters in metadata, so I have a try + catch wrapper to allow it to continue and notify us of weird characters

In [165]:
%%time 
# this takes about ~1 minute
# for vec_id in range(0, 5): # I always test it's working on 5 before jumping into the whole list
for vec_id in range(5,len(flat_df_CLMR)):
    try:
        print(vec_id)
        row = flat_df_CLMR.iloc[vec_id]
        index_clmr.update(id=str(row['vector_name']),
                        set_metadata={"genre": row['genre'],
                                      "song" : row['name'],
                                      "album": row['album'],
                                      "artists": row['artist'],
                                      "year" : str(row['year']),
                                      "clip_num" : row['vector_clip_num'],
                                      "triplet_num": str(row['embedding_triplet_num']),
                                      "spotify_id" : row['song_title']
                                  })
    except:
        print("failed on:", vec_id)
        failed_list_update_metadata_CLMR.append(vec_id)

5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279


# CLAP Uploads

In [ ]:
# Name for the index (must match that on Pinecone website)
# CLAP:
index_name_clap = 'clap-small-index'  # costs money (only activate if need be, then collapse to collection)
index_env_clap = 'us-west1-gcp'  # costs money (only activate if need be, then collapse to collection)

In [144]:
index_clap = pinecone.Index(index_name=index_name_clap)

pinecone.init(api_key=api_key, 
              environment=index_env_clap)



In [145]:
index_clap

In [146]:
pinecone.list_indexes()

['clap-small-index']

### Upload vectors to Pinecone

In [147]:
batch_id = 0
for batch_df in df_batcher(flat_df_CLAP):
    #print(batch_df)
    print("batch:", batch_id)
    batch_id = batch_id + 1
    index_clap.upsert(vectors=list(zip(batch_df["vector_name"],batch_df["flat_vector_embed_64_list"])))

batch: 0
batch: 1
batch: 2
batch: 3
batch: 4
batch: 5
batch: 6
batch: 7


### Upload their metadata as well: 

In [148]:
failed_list_update_metadata_CLAP = []
# it sometimes fails due to weird characters in metadata, so I have a try + catch wrapper to allow it to continue and notify us of weird characters

In [153]:
%%time 
# this takes about ~1 minute
# for vec_id in range(0, 5): # I always test it's working on 5 before jumping into the whole list
for vec_id in range(5,len(flat_df_CLAP)):
    try:
        print(vec_id)
        row = flat_df_CLAP.iloc[vec_id]
        index_clap.update(id=str(row['vector_name']),
                        set_metadata={"genre": row['genre'],
                                      "song" : row['name'],
                                      "album": row['album'],
                                      "artists": row['artist'],
                                      "year" : str(row['year']),
                                      "clip_num" : row['vector_clip_num'],
                                      "triplet_num": str(row['embedding_triplet_num']),
                                      "spotify_id" : row['song_title']
                                  })
    except:
        print("failed on:", vec_id)
        failed_list_update_metadata_CLAP.append(vec_id)

5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279


# Now everything is uploaded! Play around with similarity

In [168]:
p_in_ood = 'data/nathan_500_clips/not_uploaded/music/'
# find all our out of domain input files
file_list_ood = [f for f in os.listdir(p_in_ood)]


In [169]:
test_file = file_list_ood[0]

In [173]:
test_clmr_embed = DataCollection(p(p_in_ood + test_file))[0]['vecs']
test_clap_embed = model.get_audio_embedding_from_filelist(x = [p_in_ood + "/" + test_file], use_tensor=False)

In [190]:
clean_test_clmr_embed = convert_to_npfloat64_to_list(convert_to_npfloat64(flatten_vector_embed(test_clmr_embed)))
clean_test_clap_embed = convert_to_npfloat64_to_list(convert_to_npfloat64(flatten_vector_embed(test_clap_embed)))

In [191]:
index_clmr.query(
  vector=clean_test_clmr_embed,
  top_k=3,
  #include_values=False,
    include_metadata=True
)

{'matches': [{'id': 'spotify:track:5ME6skzJ9tCFJQttEvhHjC_8_0.mp3',
              'metadata': {'album': "Routes To Django & Bireli Swing '81",
                           'artists': 'Bireli Lagrene Ensemble',
                           'clip_num': '8',
                           'genre': 'gypsy jazz',
                           'song': "I've Found A New Baby",
                           'spotify_id': 'spotify:track:5ME6skzJ9tCFJQttEvhHjC',
                           'triplet_num': '1',
                           'year': '1998'},
              'score': 0.720589817,
              'values': []},
             {'id': 'spotify:track:59XAGzfCOHxdwzjmXpJrag_2_2.mp3',
              'metadata': {'album': 'Goodies',
                           'artists': 'The Howard Roberts Quartet',
                           'clip_num': '2',
                           'genre': 'jazz quartet',
                           'song': 'Girl Talk',
                           'spotify_id': 'spotify:track:59XAGzfCOHxdwzjmXp

In [192]:
index_clap.query(
  vector=clean_test_clap_embed,
  top_k=3,
  #include_values=False,
    include_metadata=True
)

{'matches': [{'id': 'spotify:track:0OtLnXYlL9nvUDhD5CCVaY_21_2.mp3',
              'metadata': {'album': 'Hoping for Love',
                           'artists': 'Isabelle Antena',
                           'clip_num': '21',
                           'genre': 'belgian modern jazz',
                           'song': 'Le Poisson Des Mers Du Sud',
                           'spotify_id': 'spotify:track:0OtLnXYlL9nvUDhD5CCVaY',
                           'triplet_num': '3',
                           'year': '1987'},
              'score': 0.86289078,
              'values': []},
             {'id': 'spotify:track:4H39zVSTrnXomaInIgfpDY_7_0.mp3',
              'metadata': {'album': 'Relaxing Bossa Lounge 3',
                           'artists': 'Various Artists',
                           'clip_num': '7',
                           'genre': 'brazilian modern jazz',
                           'song': 'You Give Me Something (Bossa Version)',
                           'spotify_id': 'spo